In [19]:
import algorithm as al
import numpy as np
import cv2 as cv
import Queue
from patchdb import *
from psi import *
from copyutils import *
from debug import *
import inspect

In [20]:
def imshow(img):
    cv.imshow('image', img)
    cv.waitKey(0)
    cv.destroyAllWindows()

In [21]:
_p = al.Inpainting()

In [22]:
_p.readImage('../../test_images/Kanizsa-triangle-tiny.png', 'source')

_p.readImage('../../test_images/Kanizsa-triangle-mask-tiny.png', 'alpha')

(True, 'Success')

In [23]:
_p._images['filled'] = np.uint8(_p._images['alpha'] > 0)*255
_p._images['inpainted'] = _p._images['source'].copy()

In [24]:
for i in range(0,3):
    _p._images['inpainted'][:,:,i] *= (_p._images['filled']>0)

#
# Step 1a,b: Identify the fill front deltaOmega and compute
#            initial patch priorities

In [25]:
_p.computeBoundaries()
_p.confidenceInitialize()

In [26]:
_p._w = 5

In [27]:
_p._patchDB = PatchDB(_p._images['inpainted'], _p._w, filled=_p._images['filled'])

In [28]:
_p.iterationsInit()
done = False

In [29]:
boundary = _p._boundaryIterator.next()
_p.fillFrontInitialize(boundary, imviewer=None)

In [30]:
boundaryPixels = boundary
# initialize the pixels on the fill front
_p._images['fillFront'] = \
    np.zeros_like(_p._images['filled'], dtype=np.uint8)
_p._images['fillFront'] = \
    cv.drawContours(_p._images['fillFront'],boundaryPixels,-1,255)

# initialize the priority queue with all points on the fill front
_p._deltaOmega = Queue.PriorityQueue()

In [31]:
col, row = boundaryPixels[10][0]
p = PSI((row, col), _p._w, 
          image=_p._images['inpainted'], 
          filled=_p._images['filled'],
          confidence=_p._images['confidence'],
          fillFront=_p._images['fillFront'])

In [32]:
cwindow, _ = copyutils.getWindow(_p._images['confidence'], p._coords, p._w)
filled, valid = copyutils.getWindow(_p._images['filled'], p._coords, p._w)
psiPArea = valid.sum()
C = cwindow[filled>0].sum() / psiPArea

In [33]:
v1 = np.array([1,2,1])
v2 = np.array([-1,0,1])
# Sobel filter for the x-derivative
sobelycol = v1[:,np.newaxis]
sobelyrow = v2[np.newaxis,:]
# Sobel filter for the y-derivative
sobelxcol = v2[:,np.newaxis]
sobelxrow = v1[np.newaxis,:]

In [34]:
pixels, _ = copyutils.getWindow(_p._images['fillFront'], p._coords, 1)

In [44]:
sobelx = np.dot(-sobelxcol, sobelxrow)
print sobelx
sobely = np.dot(-sobelycol, sobelyrow)
print sobely

[[ 1  2  1]
 [ 0  0  0]
 [-1 -2 -1]]
[[ 1  0 -1]
 [ 2  0 -2]
 [ 1  0 -1]]


In [43]:
sobely

array([[ 1,  0, -1],
       [ 2,  0, -2],
       [ 1,  0, -1]])

In [45]:
sobel7x = np.array([
    [ 3,  4,  5,  6,  5,  4,  3],
    [ 2,  3,  4,  5,  4,  3,  2],
    [ 1,  2,  3,  4,  3,  2,  1],
    [ 0,  0,  0,  0,  0,  0,  0],
    [-1, -2, -3, -4, -3, -2, -1],
    [-2, -3, -4, -5, -4, -3, -2],
    [-3, -4, -5, -6, -5, -4, -3]])

sobel7y = np.array([
    [ 3,  2,  1,  0, -1, -2, -3],
    [ 4,  3,  2,  0, -2, -3, -4],
    [ 5,  4,  3,  0, -3, -4, -5],
    [ 6,  5,  4,  0, -4, -5, -6],
    [ 5,  4,  3,  0, -3, -4, -5],
    [ 4,  3,  2,  0, -2, -3, -4],
    [ 3,  2,  1,  0, -1, -2, -3]])

In [47]:
sobel7x.shape

(7L, 7L)